In [2]:
import os
from email.parser import Parser
import email.utils

import time



In [3]:
basedir = "../maildir/arora-h"  # Change root dir to affect how many mails are touched

In [5]:
def messageIDtoSubject(mail_dict, messageID):
    return mail_dict[messageID]["subject"].replace(" ", "")


def raw_parse(inputfile, email_list):
    with open(inputfile, "r",encoding="utf-8",errors="ignore") as f:
        data = f.read()
    parsedEmail = Parser().parsestr(data)
    timestamp = time.mktime(email.utils.parsedate(parsedEmail["date"]))
    email_list.append((timestamp, parsedEmail))


def obtain_raw_threads(mail_dict, email_list):
    subject_dict = {}
    rootMailSet = set()
    auxiliarRootStructure = {}

    for mail in email_list:
        actualEmail = mail[1]
        true_subject = actualEmail['subject']
        true_subject = true_subject.replace(" ", "")
        mail_dict[actualEmail["message-id"]] = actualEmail
        if "Re:" != true_subject[0:3]:
            auxiliarRootStructure[true_subject] = actualEmail["message-id"]
            if true_subject not in subject_dict:
                subject_dict[true_subject] = []
        else:
            new_subject = true_subject.replace("Re:", "")
            if new_subject in subject_dict:
                subject_dict[new_subject].append(actualEmail["message-id"])
                rootMailSet.add(auxiliarRootStructure[new_subject])
            subject_dict[true_subject] = []
    threads = {}
    for mail in rootMailSet:
        subject = messageIDtoSubject(mail_dict, mail)
        childThread = subject_dict[subject]
        threads[mail] = childThread
    return threads

In [6]:
mail_dict = {}
email_list = []
#anal= analyzer.freelingAnalyzer()
for directory, subdirectory, filenames in os.walk(basedir):
    for filename in filenames:
        raw_parse(os.path.join(directory, filename), email_list)
email_list.sort(key=lambda x: x[0])
pureThreads = obtain_raw_threads(mail_dict, email_list)

In [15]:
email_list[0]

(315500400.0, <email.message.Message at 0x7f58a4301c30>)

In [14]:
print(email_list[0][1].get_payload())

Pushkar

I quickly plotted the short and the long curves and the following come out in 
your face.

1. Is this policy easing going to be a one shot ( a la 98 ) followed by a 
quick pick up in economic activity, or is this going to be a more deeper 
situation. That is the big one.  The curves vote for a small road-bump which 
will be rectified by a couple of eases over the next 6 months. Something the 
Fed can handle easily and quickly. This is most likely NOT going to happen - 
this is not a 98 and things are going to play out differently.  There are a 
number of other possibilities.

2. The current slow-down is two pronged - capital spending slowdown from 
businesses ( especially on IT with the internet bust ) and the consumer 
spending slow-down , possibly because of the stock market wealth effect. Both 
of these have already hit the durable goods section.  This reversal is 'Real' 
which takes longer to turn then the 'Financial crisis' hence there is good 
possibility we see a protra

In [16]:
import io

In [33]:
foo = email_list[0][1].get_payload()
stri = io.StringIO(foo)

In [34]:
lin = stri.readline()
while lin != '':
    print(lin)
    lin = stri.readline()

Pushkar



I quickly plotted the short and the long curves and the following come out in 

your face.



1. Is this policy easing going to be a one shot ( a la 98 ) followed by a 

quick pick up in economic activity, or is this going to be a more deeper 

situation. That is the big one.  The curves vote for a small road-bump which 

will be rectified by a couple of eases over the next 6 months. Something the 

Fed can handle easily and quickly. This is most likely NOT going to happen - 

this is not a 98 and things are going to play out differently.  There are a 

number of other possibilities.



2. The current slow-down is two pronged - capital spending slowdown from 

businesses ( especially on IT with the internet bust ) and the consumer 

spending slow-down , possibly because of the stock market wealth effect. Both 

of these have already hit the durable goods section.  This reversal is 'Real' 

which takes longer to turn then the 'Financial crisis' hence there is good 

possibili

In [29]:
for line in stri:
    print(line)

Pushkar



I quickly plotted the short and the long curves and the following come out in 

your face.



1. Is this policy easing going to be a one shot ( a la 98 ) followed by a 

quick pick up in economic activity, or is this going to be a more deeper 

situation. That is the big one.  The curves vote for a small road-bump which 

will be rectified by a couple of eases over the next 6 months. Something the 

Fed can handle easily and quickly. This is most likely NOT going to happen - 

this is not a 98 and things are going to play out differently.  There are a 

number of other possibilities.



2. The current slow-down is two pronged - capital spending slowdown from 

businesses ( especially on IT with the internet bust ) and the consumer 

spending slow-down , possibly because of the stock market wealth effect. Both 

of these have already hit the durable goods section.  This reversal is 'Real' 

which takes longer to turn then the 'Financial crisis' hence there is good 

possibili

In [26]:

while True:
    nl = stri.readline()
    if nl != '':
        print(nl.strip('\n'))

Pushkar

I quickly plotted the short and the long curves and the following come out in 
your face.

1. Is this policy easing going to be a one shot ( a la 98 ) followed by a 
quick pick up in economic activity, or is this going to be a more deeper 
situation. That is the big one.  The curves vote for a small road-bump which 
will be rectified by a couple of eases over the next 6 months. Something the 
Fed can handle easily and quickly. This is most likely NOT going to happen - 
this is not a 98 and things are going to play out differently.  There are a 
number of other possibilities.

2. The current slow-down is two pronged - capital spending slowdown from 
businesses ( especially on IT with the internet bust ) and the consumer 
spending slow-down , possibly because of the stock market wealth effect. Both 
of these have already hit the durable goods section.  This reversal is 'Real' 
which takes longer to turn then the 'Financial crisis' hence there is good 
possibility we see a protra

KeyboardInterrupt: 

In [ ]:
def messageIDtoSubject(mail_dict, messageID):
    return mail_dict[messageID]["subject"].replace(" ", "")


def raw_parse(inputfile, email_list):
    with open(inputfile, "r",encoding="utf-8",errors="ignore") as f:
        data = f.read()
    parsedEmail = Parser().parsestr(data)
    timestamp = time.mktime(email.utils.parsedate(parsedEmail["date"]))
    email_list.append((timestamp, parsedEmail))


def obtain_raw_threads(mail_dict, email_list):
    subject_dict = {}
    rootMailSet = set()
    auxiliarRootStructure = {}

    for mail in email_list:
        actualEmail = mail[1]
        true_subject = actualEmail['subject']
        true_subject = true_subject.replace(" ", "")
        mail_dict[actualEmail["message-id"]] = actualEmail
        if "Re:" != true_subject[0:3]:
            auxiliarRootStructure[true_subject] = actualEmail["message-id"]
            if true_subject not in subject_dict:
                subject_dict[true_subject] = []
        else:
            new_subject = true_subject.replace("Re:", "")
            if new_subject in subject_dict:
                subject_dict[new_subject].append(actualEmail["message-id"])
                rootMailSet.add(auxiliarRootStructure[new_subject])
            subject_dict[true_subject] = []
    threads = {}
    for mail in rootMailSet:
        subject = messageIDtoSubject(mail_dict, mail)
        childThread = subject_dict[subject]
        threads[mail] = childThread
    return threads


mail_dict = {}
email_list = []
anal= analyzer.freelingAnalyzer()
for directory, subdirectory, filenames in os.walk(basedir):
    for filename in filenames:
        raw_parse(os.path.join(directory, filename), email_list)
email_list.sort(key=lambda x: x[0])
pureThreads = obtain_raw_threads(mail_dict, email_list)
anal.setup()
print(email_list[0][1].get_payload())
anal.process(email_list[0][1].get_payload())
